1 define path

In [76]:
import os
from glob import glob

In [57]:
homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/reduction'
obsid = '0105860101'
os.chdir(homepath)

2 Initialize steps

In [5]:
f = open('cifbuild.sh', 'w')
f.write(
f'''
export SAS_ODF={homepath}
cifbuild 2>&1 | tee cifbuild.log

'''
)
f.close()




In [8]:
f = open('odfingest.sh', 'w')
f.write(f'''
export SAS_CCF={homepath}/ccf.cif
export SAS_ODF={homepath}
odfingest odfdir=$SAS_ODF outdir=$SAS_ODF 2>&1 | tee odfingest.log
''')
f.close()

3 Set SAS environment

In [77]:
homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/reduction/SDSSTG3460'

f = open('set_sas.txt', 'w')
sasfile= glob('*SUM.SAS')[0] 
f.write(f'''
export SAS_CCF={homepath}/ccf.cif
export SAS_ODF={homepath}/{sasfile}
sasversion
''')
f.close()

4 emproc

In [14]:
f = open('emproc.sh', 'w')
f.write(
'''
emproc 2>&1 |tee emproc.log
mv *EMOS1_S001_ImagingEvts.ds mos1-S001.evt
mv *EMOS2_S002_ImagingEvts.ds mos2-S002.evt
'''
)
f.close()

5 epproc

In [15]:
f = open('epproc.sh', 'w')
f.write(
'''
epproc 2>&1 |tee epproc.log
mv *ImagingEvts.ds pn-S003.evt
# epproc withoutoftime=true 2>&1 |tee epproc_oot.log
# mv *EPN_S003_ImagingEvts.ds pn-oot-S003.evt
'''
)
f.close()

6 emanom: diagnose abnormal bright chips in MOS
NO NEED! the esas has run the *-anom.log!

In [18]:
f = open('emanom.sh', 'w')
f.write(
'''
emanom eventfile=mos1-S001.evt keepcorner=no 2>&1 | tee emanom_mos1.log
emanom eventfile=mos2-S002.evt keepcorner=no 2>&1 | tee emanom_mos2.log
'''
)
f.close()

# mos2 chip5 is I, but not serious by visually inspect, not excluded. 
# This observation is indeed contaminated by SP. (40% exp time is flared)


In [17]:
f = open('set_chips_on.txt', 'w')
f.write(
'''
export M1ON="T T T T T T T"
export M2ON="T T T T T T T"
export PNON="T T T T"
'''
)
f.close()

In [21]:
# visual inspect
f = open('chips_visual_inspect.sh', 'w')
f.write(
'''
evselect table=mos1-S001.evt withimageset=yes imageset=mos1S001-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&&(PATTERN<=12)&&((FLAG & 0x766aa000)==0)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
evselect table=mos2-S002.evt withimageset=yes imageset=mos2S002-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&& (PATTERN<=12)&&((FLAG & 0x766aa000)==0)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
evselect table=pn-S003.evt withimageset=yes imageset=pnS003-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&& (PATTERN <= 4)&&(#XMMEA_EP)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
evselect table=pn-oot-S003.evt withimageset=yes imageset=pnS003-oot-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&& (PATTERN <= 4)&&(#XMMEA_EP)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
'''
)
f.close()


7 SP flare filtering

In [24]:
f = open('sp_flare.sh', 'w')
f.write(
'''
espfilt eventfile=mos1-S001.evt elow=2500 ehigh=8500 withsmoothing=yes smooth=51 rangescale=6.0 allowsigma=3.0 method=histogram keepinterfiles=false 2>&1 | tee flare_filter_mos1-S001.log
espfilt eventfile=mos2-S002.evt elow=2500 ehigh=8500 withsmoothing=yes smooth=51 rangescale=6.0 allowsigma=3.0 method=histogram keepinterfiles=false 2>&1 | tee flare_filter_mos2-S002.log
espfilt eventfile=pn-S003.evt elow=2500 ehigh=8500 withsmoothing=yes smooth=51 rangescale=15.0 allowsigma=3.0 method=histogram withoot=Y ootfile=pn-oot-S003.evt keepinterfiles=false 2>&1 | tee flare_filter_pn-S003.log
'''
)

f.close()
# except for P-allevc.fits and P-gti.fits are kept, others are moved to DIAGNOISE dir after checking


8 check sp flare filtering diagnostic files
Files: P-hist.qdp
Conclusion: 
not very flared


9 Remove the point sources: go to pointsources-pipeline.ipynb

10 Convert the annuli region from fk5 coords to detXY

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

def deg2sex(ra_deg, dec_deg):
    # Create a SkyCoord object with the specified units
    coords = SkyCoord(ra=ra_deg * u.degree, dec=dec_deg * u.degree)
    # Extract the converted sexagesimal values
    ra_sexagesimal = coords.to_string(style='hmsdms', precision=2)
    return f"{ra_sexagesimal.split(' ')[0]}", f"{ra_sexagesimal.split(' ')[1]}"

In [78]:
# convert the center coords to det XY

# read the fk5 eckert region
datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/eckert/0105860101/region'
file = glob(f'{datapath}/RGH80_reg0.reg')
f = open(file)
line = f.readlines()[-1]
ra = line.split('(')[-1].split(',')[0]
dec = line.split(',')[1]
f.close()

# write the command to convert 
f = open('deg2det.sh', 'w')

f.write(
f'''
export M1=mos1S001
export M2=mos2S002
export PN=pnS003

esky2det datastyle=user ra={ra} dec={dec} outunit=det calinfoset='mos1S001-allevc-exchips.fits' 2>&1 | tee deg2det-$M1.log
esky2det datastyle=user ra={ra} dec={dec} outunit=det calinfoset='mos2S002-allevc-exchips.fits' 2>&1 | tee deg2det-$M2.log
esky2det datastyle=user ra={ra} dec={dec} outunit=det calinfoset='pnS003-allevc-exchips.fits' 2>&1 | tee deg2det-$PN.log
''')

f.close()


In [46]:
''' RGH 80
   307.6     -183.9
   15.1       90.4
   132.2     -235.2

'''

detx_dict = {'mos1S001':307.6, 'mos2S002':15.1, 'pnS003':132.2}
dety_dict = {'mos1S001':-183.9, 'mos2S002':90.4, 'pnS003':-235.2}

In [4]:
print(os.getcwd())
'''ID3460
294.8     -203.6
34.7       77.5
112.5     -222.4


'''

detx_dict = {'mos1S001':294.8 , 'mos2S002':34.7, 'pnS003':112.5}
dety_dict = {'mos1S001':-203.6, 'mos2S002':77.5, 'pnS003':-222.4}

/Users/eusracenorth/Documents/work/XGAP-ABUN/codes/ESAS


In [10]:
# read the fk5 eckert region
from glob import glob
import numpy as np


datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/eckert/0105860101/regions'
files = glob(f'{datapath}/RGH80_reg*.reg')

reg_radius = []
for file in files:
    f = open(file)
    line = f.readlines()[-1]
    reg_radius.append(float(line.split(',')[-1].split("'")[0]))
    f.close()

reg_radius = np.unique(reg_radius)


[ 0.25     0.55     0.91     1.342    1.8604   2.48248  3.22898  4.12477
  5.19973  6.48967  8.0376   9.89513 12.1242 ]


In [50]:
## create the region files
os.makedirs(f'{homepath}/spec_reg')
os.chdir(f'{homepath}/spec_reg')

r500 = 7.35216
bkg_r2 = 14.799


for name in ['mos1S001', 'mos2S002', 'pnS003']: # 
    for i in range(1, len(reg_radius)):
        f = open(f'reg{i}-{name}.reg', 'w')
        f.write(
f'''&&((DETX,DETY) IN annulus({detx_dict[name]},{dety_dict[name]},{(reg_radius[int(i-1)] * 60/0.05):.2f},{(reg_radius[i]* 60/0.05):.2f}))''')
    
        f.close()

    f = open(f'reg0-{name}.reg', 'w')
    f.write(
f'''&&((DETX,DETY) IN circle({detx_dict[name]},{dety_dict[name]},{(reg_radius[0]*60/0.05):.2f}))
''')
    f.close()

    f = open(f'reg{len(reg_radius)}-{name}.reg', 'w')
    f.write(
f'''&&((DETX,DETY) IN circle({detx_dict[name]},{dety_dict[name]},{(reg_radius[-1]*60/0.05):.2f}))
''')
    f.close()

    f = open(f'R500-{name}.reg', 'w')
    f.write(
f'''&&((DETX,DETY) IN circle({detx_dict[name]},{dety_dict[name]},{(r500*60/0.05):.2f}))
''')
    f.close()

    f = open(f'bkg-{name}.reg', 'w')
    f.write(
f'''&&((DETX,DETY) IN annulus({detx_dict[name]},{dety_dict[name]},{(reg_radius[-1]*60/0.05):.2f},{(bkg_r2*60/0.05):.2f}))
''')
    f.close()





13 mosspectra mosback

In [51]:
print(os.getcwd())
f = open('mosspectra_annu.sh', 'w')
f.write(
'''
source set_chips_on.txt
source set_sas.txt

export ELO=700
export EHI=1200
export M1=mos1S001
export M2=mos2S002

rm mosspectra_${M1}.log
rm mosspectra_${M2}.log

for regfile in spec_reg/bkg-mos1S001.reg
do
regname=$(basename $regfile | cut -d'-' -f 1)
echo $regname

mosspectra eventfile=${M1}-allevc.fits keepinterfiles=yes withregion=yes regionfile=spec_reg/${regname}-${M1}.reg pattern=12 withsrcrem=yes maskdet=${M1}-bkgregtdet.fits masksky=${M1}-bkgregtsky.fits elow=${ELO} ehigh=${EHI} ccds="${M1ON}" -V=7 >> mosspectra_${M1}.log 2>&1
mosback inspecfile=${M1}-fovt.pi elow=${ELO} ehigh=${EHI} ccds="${M1ON}" 2>&1 |tee mosback_${M1}.log
mosspectra eventfile=${M2}-allevc.fits keepinterfiles=yes withregion=yes regionfile=spec_reg/${regname}-${M1}.reg pattern=12 withsrcrem=yes maskdet=${M2}-bkgregtdet.fits masksky=${M2}-bkgregtsky.fits elow=${ELO} ehigh=${EHI} ccds="${M2ON}" -V=7 >> mosspectra_${M2}.log 2>&1
mosback inspecfile=${M2}-fovt.pi elow=${ELO} ehigh=${EHI} ccds="${M2ON}" 2>&1 |tee mosback_${M2}.log

for name in $M1 $M2
do
mkdir ${regname}
mkdir ${regname}/img
mkdir ${regname}/diagnose
mv ${name}*-${ELO}-${EHI}* ${regname}/img
mv ${name}*.pi ${regname}/spec
mv ${name}*.qdp ${regname}/diagnose
mv ${name}*imt* ${regname}/img
mv ${name}*.arf ${regname}
mv ${name}*.rmf ${regname}
mv ${name}*imspdet* ${regname}/img

cp ${regname}/spec/${name}-fovt.pi ${regname}

done
done
'''
)
f.close()

/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/reduction/spec_reg/spec_reg


14 pnspectra, pnback

In [58]:
f = open('pnspectra_annu.sh', 'w')

f.write('''
source set_chips_on.txt
source set_sas.txt

export PN=pnS003
export ELO=700
export EHI=1200

for pat in 0 4
do

rm pnspectra_${pat}_${PN}.log
rm pnback_${pat}_${PN}.log

for regfile in spec_reg/bkg-mos1S001.reg
do
regname=$(basename $regfile | cut -d'-' -f 1)
echo $regname

pnspectra eventfile=${PN}-allevc.fits ootevtfile=${PN}-allevcoot.fits keepinterfiles=yes withregion=yes regionfile=spec_reg/${regname}-${PN}.reg pattern=${pat} withsrcrem=yes maskdet=${PN}-bkgregtdet.fits masksky=${PN}-bkgregtsky.fits elow=${ELO} ehigh=${EHI} quads="${PNON}" -V=7 >> pnspectra_${pat}_${PN}.log 2>&1 
pnback inspecfile=${PN}-fovt.pi inspecoot=${PN}-fovtoot.pi elow=${ELO} ehigh=${EHI} quads="${PNON}" >> pnback_${pat}_${PN}.log 2>&1 


mkdir ${regname}/${PN}_${pat}
mkdir ${regname}/${PN}_${pat}/img
mkdir ${regname}/${PN}_${pat}/diagnose

mv pn*-${ELO}-${EHI}* ${regname}/${PN}_${pat}/img
mv pn*.pi ${regname}/${PN}_${pat}_spec
mv pn*.qdp ${regname}/${PN}_${pat}/diagnose
mv pn*imt* ${regname}/${PN}_${pat}/img
mv pn*.arf ${regname}/${PN}-${pat}-fovt.arf
mv pn*.rmf ${regname}/${PN}-${pat}-fovt.rmf
mv pn*imspdet* ${regname}/${PN}_${pat}/img

cp ${regname}/${PN}_${pat}/${PN}-fovt.pi ${regname}/${PN}-${pat}-fovt.pi
cp ${regname}/${PN}_${pat}/${PN}-fovtoot.pi ${regname}/${PN}-${pat}-fovtoot.pi

done
done
'''
)
f.close()